# Demand for storables
In this example we model the demand for storable goods. For a peer reviewed model
close to the formulation here, see Nevo, but this simpler version is due
to a presentation by Günter J. Hitsch (2013).

If prices for storable goods vary over time, it allows consumers to buy when
prices are low, and gradually consume from their inventories, to avoid buying
when the prices are high. For simplicity, assume that there are three choices:
don't buy ($k=0$), buy a small package with 1 unit ($k=1$), or buy a large package with two units ($k=2$).
The prices of the packages, $P_k$, follow a Markov process. The inventory evolution
is determined by the number of packages bought, $n_k$,  and the number of packages
consumed. Say one unit is consumed per period, and $I$ is the upper bound on the
storage in the household, then the inventory, $i$, evolves according to
$$
i' = \begin{cases}0&\text{if } i+n_k=0\\i+n_k-1&\text{if } 1\leq i+n_k\leq I\\I&\text{if } i+n_k>I\end{cases}
$$
In code this is

In [1]:
next_i(i, k) = max.(0, min.(I, i+n[k]-1))

next_i (generic function with 1 method)

The utility of buying a package of size $k$ is
$$
-\alpha P_k - \tau(n_k)
$$
where $P_k$ is the price of package size $k$, and $n_k$ is the number of units in
package size $k$. We take $n_1 = 2$ and $n_2=5$. $\tau(\cdot)$ is the transportation
cost given the number of units to transport. Additionally, there is a storage cost
$c(i')$. This implies the following utility function
$$
u(k~|~x) = \begin{cases}\delta-\alpha P_k - \tau(n_k)-c(i') & \text{if } k>0\\\delta-c(i') & \text{if } k=0 \wedge i\geq 0\\0&\text{if } k=0\wedge i=0\end{cases}
$$
Let us assume that consumers can hold up to $20$ units of goods, and that the prices
are $P_\text{low}=(1.2,3.0)$, and $P_\text{high}=(2.0,5.0)$. As is clear, the unit prices
are identical, but buying the large package allows you to stockpile in the cheap period.

Looking at the price dimension of the state space we have a transition matrix
$$
\begin{pmatrix}0.16 & 0.84\\
               0.16 & 0.84
               \end{pmatrix}
$$
Assume parameter values $\delta = 4$, $\alpha=4$, $c(i)=0.05\cdot i$, and $\tau(n_k)=0$. Translating this to code we get

In [2]:
using StocasEstimators

In [3]:
F_X = ["low", "high"]
F_P = [0.16 0.84; 0.16 0.84]
SP = CommonState("price", F_X, F_P)

Stocas.CommonState{String,Array{String,1},Array{Array{Float64,2},1}}("price", String["low", "high"], Array{Float64,2}[[0.16 0.84; 0.16 0.84]], 2)

We encode this as a common state, because the prices apply to all costumers.
Now define the individual inventory state

In [4]:
I = 20
n = [0, 2, 5]
K = length(n)
F_i = [spzeros(1+I, 1+I) for k = 1:K]
for i = 0:I, j = 1:K
        F_i[j][i+1, next_i(i, j)+1]=1
end
Si = State("inventory", 0:I, F_i)

Discrete state


Finally, we combine the two states as usual

In [5]:
S = States(SP, Si)

 * name: inventory
 * n: 21


Now define consumption utility (from the unit consumption) and the price sensitivity to be

In [6]:
delta, alpha = 4., 4.

Discrete states
 * Number of state variables: 2
 * Total number of states:    42
 * Individual states:           
   1) price (n: 2)
   2) inventory (n: 21)


(4.0, 4.0)

Lastly, we construct the $\texttt{LinearUtility}$ instance

In [7]:
Z1 = [[zeros(1); ones(I)] zeros(I+1) -next_i(0:I, 1);
      [zeros(1); ones(I)] zeros(I+1) -next_i(0:I, 1)]
Z2 = [ones(2*(I+1)) -[1.2*ones(I+1);2*ones(I+1)] -kron(ones(2), next_i(0:I, 2))]
Z3 = [ones(2*(I+1)) -[3.0*ones(I+1);5*ones(I+1)] -kron(ones(2), next_i(0:I, 3))]
U = LinearUtility(("Buy 0", "Buy 2", "Buy 5"),
                  (Z1, Z2, Z3), 0.998, [delta; alpha; 0.05])

Stocas.LinearUtility{3,Float64}(("Buy 0", "Buy 2", "Buy 5"), ([0.0 0.0 0.0; 1.0 0.0 0.0; … ; 1.0 0.0 -18.0; 1.0 0.0 -19.0], [1.0 -1.2 -1.0; 1.0 -1.2 -2.0; … ; 1.0 -2.0 -20.0; 1.0 -2.0 -20.0], [1.0 -3.0 -4.0; 1.0 -3.0 -5.0; … ; 1.0 -5.0 -20.0; 1.0 -5.0 -20.0]), 0.998, [4.0, 4.0, 0.05], ([0.0, 4.0, 3.95, 3.9, 3.85, 3.8, 3.75, 3.7, 3.65, 3.6  …  3.5, 3.45, 3.4, 3.35, 3.3, 3.25, 3.2, 3.15, 3.1, 3.05], [-0.85, -0.9, -0.95, -1.0, -1.05, -1.1, -1.15, -1.2, -1.25, -1.3  …  -4.6, -4.65, -4.7, -4.75, -4.8, -4.85, -4.9, -4.95, -5.0, -5.0], [-8.2, -8.25, -8.3, -8.35, -8.4, -8.45, -8.5, -8.55, -8.6, -8.65  …  -16.75, -16.8, -16.85, -16.9, -16.95, -17.0, -17.0, -17.0, -17.0, -17.0]), ([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]), 3)

We are now ready to solve and simulate from the model.

This model is different from the other models as it has three choices. However, nothing changes
with regards to the solution interface.

In [8]:
Vvfi, itersvfi = solve(U, S, Stocas.VFI())
Vnewt, itersnewt = solve(U, S, Stocas.Newton())
Vpoli, iterspoli = solve(U, S, Stocas.Policy())
Vpoly, iterspoly = solve(U, S, Stocas.Poly())

(Stocas.IntegratedValueFunction{Array{Float64,1},Array{Float64,2},Array{Array{Float64,1},1}}([629.544, 632.44, 635.244, 637.969, 640.625, 643.216, 645.747, 648.219, 650.632, 652.987  …  656.869, 659.134, 661.329, 663.454, 665.514, 667.509, 669.44, 671.307, 673.113, 674.852], [629.544, 632.44, 635.244, 637.969, 640.625, 643.216, 645.747, 648.219, 650.632, 652.987  …  656.869, 659.134, 661.329, 663.454, 665.514, 667.509, 669.44, 671.307, 673.113, 674.852], [0.00102322 0.0162845 … 0.0 0.0; 0.00308588 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.00839406; 0.0 0.0 … 0.836846 0.0014743], Array{Float64,1}[[624.494, 624.494, 628.111, 631.465, 634.626, 637.629, 640.5, 643.256, 645.91, 648.47  …  653.34, 655.659, 657.907, 660.085, 662.197, 664.244, 666.227, 668.145, 670.003, 671.801], [628.111, 631.465, 634.626, 637.629, 640.5, 643.256, 645.91, 648.47, 650.945, 653.34  …  657.907, 660.085, 662.197, 664.244, 666.227, 668.145, 670.003, 671.801, 673.509, 673.509], [637.629, 640.5, 643.256, 645.91, 648.47, 65

In [9]:
using Plots
pgfplots()
plot(("low", :), S, U, :policy, title="Purchase probabilities for different quantities given prices are low.")
# savefig("hitsch-fig.pdf")

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 
 
 
 
<path style="stroke:none;" d="M 4.578125 -3.1875 C 4.578125 -3.984375 4.53125 -4.78125 4.1875 -5.515625 C 3.734375 -6.484375 2.90625 -6.640625 2.5 -6.640625 C 1.890625 -6.640625 1.171875 -6.375 0.75 -5.453125 C 0.4375 -4.765625 0.390625 -3.984375 0.390625 -3.1875 C 0.390625 -2.4375 0.421875 -1.546875 0.84375 -0.78125 C 1.265625 0.015625 2 0.21875 2.484375 0.21875 C 3.015625 0.21875 3.78125 0.015625 4.21875 -0.9375 C 4.53125 -1.625 4.578125 -2.40625 4.578125 -3.1875 Z M 2.484375 0 C 2.09375 0 1.5 -0.25 1.328125 -1.203125 C 1.21875 -1.796875 1.21875 -2.71875 1.21875 -3.3125 C 1.21875 -3.953125 1.21875 -4.609375 1.296875 -5.140625 C 1.484375 -6.328125 2.234375 -6.421875 2.484375 -6.421875 C 2.8125 -6.421875 3.46875 -6.234375 3.65625 -5.25 C 3.765625 -4.6875 3.765625 -3.9375 3.765625 -3.3125 C 3.765625 -2.5625 3.765625 -1.890625 3.65625 -1.25 C 3.5 -0.296875 2.9375 0 2.484375 0 Z M 2.484375 0 "/>
 
 
<path style="stroke:none;" d="M 1.265625 -0.765625 L 2.328125 -1.796875 C 3.875 -3.171875 4.46875 -3.703125 4.46875 -4.703125 C 4.46875 -5.84375 3.578125 -6.640625 2.359375 -6.640625 C 1.234375 -6.640625 0.5 -5.71875 0.5 -4.828125 C 0.5 -4.28125 1 -4.28125 1.03125 -4.28125 C 1.203125 -4.28125 1.546875 -4.390625 1.546875 -4.8125 C 1.546875 -5.0625 1.359375 -5.328125 1.015625 -5.328125 C 0.9375 -5.328125 0.921875 -5.328125 0.890625 -5.3125 C 1.109375 -5.96875 1.65625 -6.328125 2.234375 -6.328125 C 3.140625 -6.328125 3.5625 -5.515625 3.5625 -4.703125 C 3.5625 -3.90625 3.078125 -3.125 2.515625 -2.5 L 0.609375 -0.375 C 0.5 -0.265625 0.5 -0.234375 0.5 0 L 4.203125 0 L 4.46875 -1.734375 L 4.234375 -1.734375 C 4.171875 -1.4375 4.109375 -1 4 -0.84375 C 3.9375 -0.765625 3.28125 -0.765625 3.0625 -0.765625 Z M 1.265625 -0.765625 "/>
 
 
<path style="stroke:none;" d="M 2.9375 -1.640625 L 2.9375 -0.78125 C 2.9375 -0.421875 2.90625 -0.3125 2.171875 -0.3125 L 1.96875 -0.3125 L 1.96875 0 C 2.375 -0.03125 2.890625 -0.03125 3.3125 -0.03125 C 3.734375 -0.03125 4.25 -0.03125 4.671875 0 L 4.671875 -0.3125 L 4.453125 -0.3125 C 3.71875 -0.3125 3.703125 -0.421875 3.703125 -0.78125 L 3.703125 -1.640625 L 4.6875 -1.640625 L 4.6875 -1.953125 L 3.703125 -1.953125 L 3.703125 -6.484375 C 3.703125 -6.6875 3.703125 -6.75 3.53125 -6.75 C 3.453125 -6.75 3.421875 -6.75 3.34375 -6.625 L 0.28125 -1.953125 L 0.28125 -1.640625 Z M 2.984375 -1.953125 L 0.5625 -1.953125 L 2.984375 -5.671875 Z M 2.984375 -1.953125 "/>
 
 
<path style="stroke:none;" d="M 1.3125 -3.265625 L 1.3125 -3.515625 C 1.3125 -6.03125 2.546875 -6.390625 3.0625 -6.390625 C 3.296875 -6.390625 3.71875 -6.328125 3.9375 -5.984375 C 3.78125 -5.984375 3.390625 -5.984375 3.390625 -5.546875 C 3.390625 -5.234375 3.625 -5.078125 3.84375 -5.078125 C 4 -5.078125 4.3125 -5.171875 4.3125 -5.5625 C 4.3125 -6.15625 3.875 -6.640625 3.046875 -6.640625 C 1.765625 -6.640625 0.421875 -5.359375 0.421875 -3.15625 C 0.421875 -0.484375 1.578125 0.21875 2.5 0.21875 C 3.609375 0.21875 4.5625 -0.71875 4.5625 -2.03125 C 4.5625 -3.296875 3.671875 -4.25 2.5625 -4.25 C 1.890625 -4.25 1.515625 -3.75 1.3125 -3.265625 Z M 2.5 -0.0625 C 1.875 -0.0625 1.578125 -0.65625 1.515625 -0.8125 C 1.328125 -1.28125 1.328125 -2.078125 1.328125 -2.25 C 1.328125 -3.03125 1.65625 -4.03125 2.546875 -4.03125 C 2.71875 -4.03125 3.171875 -4.03125 3.484375 -3.40625 C 3.65625 -3.046875 3.65625 -2.53125 3.65625 -2.046875 C 3.65625 -1.5625 3.65625 -1.0625 3.484375 -0.703125 C 3.1875 -0.109375 2.734375 -0.0625 2.5 -0.0625 Z M 2.5 -0.0625 "/>
 
 
<path style="stroke:none;" d="M 1.625 -4.5625 C 1.171875 -4.859375 1.125 -5.1875 1.125 -5.359375 C 1.125 -5.96875 1.78125 -6.390625 2.484375 -6.390625 C 3.203125 -6.390625 3.84375 -5.875 3.84375 -5.15625 C 3.84375 -4.578125 3.453125 -4.109375 2.859375 -3.765625 Z M 3.078125 -3.609375 C 3.796875 -3.984375 4.28125 -4.5 4.28125 -5.15625 C 4.28125 -6.078125 3.40625 -6.640625 2.5 -6.640625 C 1.5 -6.640625 0.6875 -5.90625 0.6875 -4.96875 C 0.6875 -4.796875 0.703125 -4.34375 1.1

# References
Nevo
Günter J. Hitsch, "Single Agent Dynamics:  Dynamic Discrete Choice", The University of Chicago Booth School of Business, 2013.

In [11]:
T, N = 120, 50
D = simulate(U, S, 1, T, N)

Stocas.Data{Int64}([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  120, 120, 120, 120, 120, 120, 120, 120, 120, 120], [3, 1, 1, 1, 1, 2, 1, 1, 2, 2  …  2, 3, 1, 1, 1, 1, 1, 1, 2, 1], Array{Int64,1}[[2, 3, 4, 5, 7, 8, 11, 12, 14, 15  …  5988, 5989, 5990, 5993, 5994, 5995, 5996, 5997, 5998, 6000], [6, 9, 10, 13, 16, 24, 27, 34, 44, 46  …  5941, 5949, 5953, 5961, 5964, 5967, 5974, 5979, 5991, 5999], [1, 20, 22, 29, 42, 51, 72, 74, 77, 78  …  5901, 5920, 5924, 5933, 5951, 5970, 5972, 5976, 5983, 5992]], [1, 26, 25, 24, 23, 22, 23, 22, 22, 23  …  25, 5, 30, 29, 28, 27, 26, 25, 24, 25], [1 1; 2 5; … ; 2 3; 2 4], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  30, 31, 32, 33, 34, 35, 36, 37, 38, 39], [3 21 180; 3 16 55; … ; 9 0 0; 2 0 0], 6000)

In [12]:
nfxp_results = fit_nfxp(U, S, D)
npl_results = fit_npl(U, S, D)


LoadError: DomainError:
log will only return a complex result if called with a complex argument. Try log(complex(x)).

In [13]:
D.nxjᵈ

35×3 Array{Int64,2}:
   3  21  180
   3  16   55
   4  10   30
   7  31   71
   9  17   53
  12  28   41
  34  40   45
  33  32   32
  28  40   23
  27  37   17
  37  39   15
  35  18    8
  18   7    4
   ⋮         
 216  15    0
 233  11    0
 187  12    0
 166  12    0
 129   6    0
  97   5    0
  68   2    0
  44   1    0
  28   1    0
  17   0    0
   9   0    0
   2   0    0

In [14]:
using Plots

In [20]:
plotlyjs()
heatmap(collect(1:3), D.xᵈ, D.nxjᵈ./sum(D.nxjᵈ, 2))